In [12]:
import sqlite3
con = sqlite3.connect("youtube.db")
cur = con.cursor()

In [13]:
cur.executescript('''
    CREATE TABLE url(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc_id INTEGER NOT NULL,
    path TEXT NOT NULL,
    param TEXT,
    bj_id INTEGER NOT NULL,
    title TEXT NOT NULL
    );

    CREATE TABLE netloc(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL
    );

    CREATE TABLE chat(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    content TEXT NOT NULL,
    writer_id INTEGER NOT NULL,
    w_time TEXT NOT NULL, 
    url_id INTEGER NOT NULL
    );

    CREATE TABLE jamak(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    content TEXT NOT NULL,
    j_time TEXT NOT NULL,
    url_id INTEGER NOT NULL
    );

    CREATE TABLE bj(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL,
    netloc_id INTEGER NOT NULL
    );

    CREATE TABLE writer(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL
    );
    
    ''')

con.commit()

In [14]:
import requests
from urllib import error
import time

# 본인의 headers를 user-agent에 사용
headers = {"user-agent": }

def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:  
            print(maxretries)                                       
            time.sleep(timeout)                                     
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import time
import os
import re
import random
from urllib3.exceptions import MaxRetryError

def youtube(n = 8, seed = 'https://www.youtube.com/watch?v=9FZTL-MH6Sk&app=desktop'):

    netloc = "www.youtube.com"
    path = ["/watch"]
    param = [] 
    seed_list = []
    bj = []
    title = []
    temp_path_param = []
    temp_param = "&app=desktop"
    p = re.compile("[0-9\:\,]+\\n[ㄱ-ㅎㅏ-ㅣ가-힣0-9\,\'A-Za-z\s]+[ㄱ-ㅎㅏ-ㅣ가-힣]")
    text_temp = []
    path_temp = '/Users/user/Downloads/'
    driver = webdriver.Chrome()
    q = -1
    break_key = 0
    sub_index = 0
    
    param.append('?' + seed.split('/')[3].split('?')[1]) # 시작 seed 주소의 parameter
    
    for c in range(n):
        i = c - sub_index
        print(i)
        break_key = 0
        
        html = download("get", seed)
        dom = BeautifulSoup(html.text, "lxml")
        
        print("seed : " + seed)
        url = seed
        seed_list.append(seed)
        
        try:
            bj.append([_.text for _ in dom.select("div.yt-user-info > a")][0])
            title.append([_.text for _ in dom.select("#eow-title")][0])
        
        except IndexError: # 영상이 차단된 경우
            break_key = 1
            seed = seed_list[-2]
            sub_index += 2
            continue
        
        temp_path_param.append([_['href'] for _ in dom.select(".content-link.spf-link.yt-uix-sessionlink.spf-link")][random.randint(0,3)])
        
        path.append(temp_path_param[i].split("?")[0])
        param.append("?" + temp_path_param[i].split("?")[1] + temp_param)
        
        seed = "https://" + netloc + path[i+1] + param[i+1]
        
        cur.execute("SELECT id FROM url WHERE param=? LIMIT 0,1", [param[i]])
        temp_param_exist = cur.fetchone()
        
        if temp_param_exist:
            continue
        
        driver.get("https://savesubs.com/")
        driver.find_element_by_css_selector(".search_wrap > div").click()
        driver.find_element_by_name("url").send_keys(url)
        driver.find_element_by_css_selector(".search_wrap > input").click()
        time.sleep(15)
        try:
            for k in driver.find_elements_by_css_selector(".list-reset > li"):
                if (k.find_elements_by_css_selector("span")[0].text == "SRT") & ((k.find_elements_by_css_selector("span")[1].text == "KOREAN (AUTO-GENERATED)") | (k.find_elements_by_css_selector("span")[1].text == "KOREAN")):
                    k.find_element_by_css_selector("a").click()
                    break
            else:
                break_key = 1
                
        except IndexError: # 자막이 없는 경우
            break_key = 1
            continue
        
        if break_key:
            time.sleep(5)
            continue
        
        time.sleep(15)
        
        q += 1
        
        cur.execute("SELECT id FROM netloc WHERE netloc=? LIMIT 0,1", [netloc])
        netloc_exist = cur.fetchone()
        
        if not netloc_exist:
            cur.execute("INSERT INTO netloc(netloc) VALUES(?)", [netloc])
            con.commit()
            
            cur.execute("SELECT id FROM netloc WHERE netloc=? LIMIT 0,1", [netloc])
            netloc_exist = cur.fetchone()
        
        cur.execute("SELECT id FROM bj WHERE name=? LIMIT 0,1", [bj[i]])
        bj_exist = cur.fetchone()
        
        if not bj_exist:
            cur.execute("INSERT INTO bj(name, netloc_id) VALUES(?, ?)", [bj[i], netloc_exist[0]])
            con.commit()
                         
            cur.execute("SELECT id FROM bj WHERE name=? LIMIT 0,1", [bj[i]])
            bj_exist = cur.fetchone()
                                    
        cur.execute("SELECT id FROM url WHERE param=? LIMIT 0,1", [param[i]])
        param_exist = cur.fetchone()
        
        if not param_exist:
            cur.execute("INSERT INTO url(netloc_id, path, param, bj_id, title) VALUES(?, ?, ?, ?, ?)", 
                        [netloc_exist[0], path[i], param[i], bj_exist[0], title[i]])
            con.commit()
            
            print(title[i])
            
            cur.execute("SELECT id FROM url WHERE param=? LIMIT 0,1", [param[i]])
            param_exist = cur.fetchone()
                         
        temp = []

        for r, d, f in os.walk('/Users/user/Downloads/'):
            temp.append(f)
            # 파일을 다운로드 받는 default 경로가 /Users/user/Downloads/ 이고, 
            # Downloads 폴더 안에 어떠한 파일도 없어야 함
        
        print(temp)
        if temp[0][1] == "desktop.ini":
            filepath = path_temp + temp[0][0]
        else:
            filepath = path_temp + temp[0][1]
        print(filepath)
        f = open(filepath, encoding='UTF8')
        text = p.findall(f.read())
        f.close()
        os.remove(filepath)
        text_temp.append(text)
    
        temp1 = []
        temp2 = []

        for _ in text_temp[q]:
            temp1.extend(re.findall(r'(.+?),\d\d\d\n(.+)',_) )
        temp2.append(temp1)
        temp1 = []
        
        for x in range(len(temp2[0])):
            cur.execute("INSERT INTO jamak(content, j_time, url_id) VALUES(?, ?, ?)", 
                        [temp2[0][x][1], temp2[0][x][0], param_exist[0]])
            con.commit()
    

In [16]:
youtube(n = 500, seed = "https://www.youtube.com/watch?v=xUVM-8gvj-Q&app=desktop") # seed = 철구

# seed 주소에 &app=desktop 을 넣어주지 않으면 모바일 버전으로 바뀜

# "https://www.youtube.com/watch?v=8eSH6ZLsi6I&app=desktop" -> 차단된 영상

0
seed : https://www.youtube.com/watch?v=xUVM-8gvj-Q&app=desktop

    세계최초 택배로 X을 받았습니다...
  
[['desktop.ini', '세계최초 택배로 X을 받았습니다... - Korean (auto-generated).srt']]
/Users/user/Downloads/세계최초 택배로 X을 받았습니다... - Korean (auto-generated).srt
1
seed : https://www.youtube.com/watch?v=cwCFsHHNvOg&app=desktop

    [합방#3] [철구냄새캐리!] 내 남자의 발,배꼽,겨드랑이,정수리,입냄새 맞춰라....그리고 토하러 뛰쳐나간다?ㅣ용느x철구x기훈x지혜x가현x따규x햅번 2018.01.13
  
[['desktop.ini', '[합방#3] [철구냄새캐리!] 내 남자의 발,배꼽,겨드랑이,정수리,입냄새 맞춰라....그리고 토하러 뛰쳐나간다_ㅣ용느x철구x기훈x지혜x가현x따규x햅번 2018.01.13 - Korean (auto-generated).srt']]
/Users/user/Downloads/[합방#3] [철구냄새캐리!] 내 남자의 발,배꼽,겨드랑이,정수리,입냄새 맞춰라....그리고 토하러 뛰쳐나간다_ㅣ용느x철구x기훈x지혜x가현x따규x햅번 2018.01.13 - Korean (auto-generated).srt
2
seed : https://www.youtube.com/watch?v=HDYZcGqwbNg&app=desktop

    한국 유튜버가 일본전을 하면 생기는일 [배틀그라운드]
  
[['desktop.ini', '한국 유튜버가 일본전을 하면 생기는일 [배틀그라운드] - Korean (auto-generated).srt']]
/Users/user/Downloads/한국 유튜버가 일본전을 하면 생기는일 [배틀그라운드] - Korean (auto-generated).srt
3
seed : https://www.youtube.com/w

38
seed : https://www.youtube.com/watch?v=q80-W-dScbA&app=desktop
39
seed : https://www.youtube.com/watch?v=WHERMqnfNW4&app=desktop

    허경영의 돈의 출처를 추적한 ‘탐사보도 세븐’팀! [탐사보도 세븐 17회] 20171213
  
[['desktop.ini', '허경영의 돈의 출처를 추적한 ‘탐사보도 세븐’팀! [탐사보도 세븐 17회] 20171213 - Korean (auto-generated).srt']]
/Users/user/Downloads/허경영의 돈의 출처를 추적한 ‘탐사보도 세븐’팀! [탐사보도 세븐 17회] 20171213 - Korean (auto-generated).srt
40
seed : https://www.youtube.com/watch?v=1W9L-i5FDNg&app=desktop

    롤스로이스를 신차로 구매하면 트렁크에 뭐가 들어있을까요?
  
[['desktop.ini', '롤스로이스를 신차로 구매하면 트렁크에 뭐가 들어있을까요_ - Korean (auto-generated).srt']]
/Users/user/Downloads/롤스로이스를 신차로 구매하면 트렁크에 뭐가 들어있을까요_ - Korean (auto-generated).srt
41
seed : https://www.youtube.com/watch?v=6CrtKcQkhA4&app=desktop

    김재경 대표와 함께하는 7억 롤스로이스 팬텀과 4억 고스트의 차이점, Feat S650 Maybach
  
[['desktop.ini', '김재경 대표와 함께하는 7억 롤스로이스 팬텀과 4억 고스트의 차이점, Feat S650 Maybach - Korean (auto-generated).srt']]
/Users/user/Downloads/김재경 대표와 함께하는 7억 롤스로이스 팬텀과 4억 고스트의 차이점, Feat S650 Maybach - Korean (aut

71
seed : https://www.youtube.com/watch?v=-yeaurXhs7I&app=desktop
72
seed : https://www.youtube.com/watch?v=BYyANYJx0vI&app=desktop

    "조작없이 못사는 주작꾼들" - 20190731 엄마방송 주옥순 대표
  
[['desktop.ini', '_조작없이 못사는 주작꾼들_ - 20190731 엄마방송 주옥순 대표 - Korean (auto-generated).srt']]
/Users/user/Downloads/_조작없이 못사는 주작꾼들_ - 20190731 엄마방송 주옥순 대표 - Korean (auto-generated).srt
73
seed : https://www.youtube.com/watch?v=GhiKLBzHdbg&app=desktop

    박근혜 대통령의 옥중 구상과 내년 4월 총선/윤창중+엄익선 우담보=우리는 담대한 보수우파다! 윤창중칼럼세상TV(2019.07.31)
  
[['desktop.ini', '박근혜 대통령의 옥중 구상과 내년 4월 총선_윤창중+엄익선 우담보=우리는 담대한 보수우파다! 윤창중칼럼세상TV(2019.07.31) - Korean (auto-generated).srt']]
/Users/user/Downloads/박근혜 대통령의 옥중 구상과 내년 4월 총선_윤창중+엄익선 우담보=우리는 담대한 보수우파다! 윤창중칼럼세상TV(2019.07.31) - Korean (auto-generated).srt
74
seed : https://www.youtube.com/watch?v=Yz_NaYYBjas&app=desktop
75
seed : https://www.youtube.com/watch?v=NU_7P_tj82k&app=desktop
76
seed : https://www.youtube.com/watch?v=cjAzAhckvlg&app=desktop

    "우리를 믿어라. 일본의 소재 도발 능히 제압 가능하다" --- 반도

106
seed : https://www.youtube.com/watch?v=7rLTey3JZwM&app=desktop

    먹어본 사람만이 아는 전설적인 뭉티기맛 Raw meat [재방문]
  
[['desktop.ini', '먹어본 사람만이 아는 전설적인 뭉티기맛 Raw meat [재방문] - Korean (auto-generated).srt']]
/Users/user/Downloads/먹어본 사람만이 아는 전설적인 뭉티기맛 Raw meat [재방문] - Korean (auto-generated).srt
107
seed : https://www.youtube.com/watch?v=TjJ5E1vx4ws&app=desktop
108
seed : https://www.youtube.com/watch?v=nc-cGm78chU&app=desktop

    10kg 대방어회!! 기본손질부터 회떠서 먹기까지 전체 영상!!
  
[['10kg 대방어회!! 기본손질부터 회떠서 먹기까지 전체 영상!! - Korean (auto-generated).srt', 'desktop.ini']]
/Users/user/Downloads/10kg 대방어회!! 기본손질부터 회떠서 먹기까지 전체 영상!! - Korean (auto-generated).srt
109
seed : https://www.youtube.com/watch?v=uMCIkaAXsKQ&app=desktop

    자연산 전복! 절대 이건 드시면 안돼요!!  모두몰랐던 전복손질법 3가지!
  
[['desktop.ini', '자연산 전복! 절대 이건 드시면 안돼요!! 모두몰랐던 전복손질법 3가지! - Korean (auto-generated).srt']]
/Users/user/Downloads/자연산 전복! 절대 이건 드시면 안돼요!! 모두몰랐던 전복손질법 3가지! - Korean (auto-generated).srt
110
seed : https://www.youtube.com/watch?v=rkwHLARGMj4&a

139
seed : https://www.youtube.com/watch?v=NfnYV_fYQ6M&app=desktop
140
seed : https://www.youtube.com/watch?v=rHjO3xjDJAo&app=desktop

    극한직업 - Extreme JOB, 멧돼지 포획단 제1부
  
[['desktop.ini', '극한직업 - Extreme JOB, 멧돼지 포획단 제1부 - Korean (auto-generated).srt']]
/Users/user/Downloads/극한직업 - Extreme JOB, 멧돼지 포획단 제1부 - Korean (auto-generated).srt
141
seed : https://www.youtube.com/watch?v=2zjTDjQoLoQ&app=desktop

    영등포지구대에 잡혀오는 사람들
  
[['desktop.ini', '영등포지구대에 잡혀오는 사람들 - Korean (auto-generated).srt']]
/Users/user/Downloads/영등포지구대에 잡혀오는 사람들 - Korean (auto-generated).srt
142
seed : https://www.youtube.com/watch?v=ywn4yDOP6ps&app=desktop

    알몸으로 도심을 누비는 남자 [현장르포 특종세상 259회]
  
[['desktop.ini', '알몸으로 도심을 누비는 남자 [현장르포 특종세상 259회] - Korean (auto-generated).srt']]
/Users/user/Downloads/알몸으로 도심을 누비는 남자 [현장르포 특종세상 259회] - Korean (auto-generated).srt
143
seed : https://www.youtube.com/watch?v=ujPKxKxtuwo&app=desktop

    임예진 이혼? 남편과 각방 생활 20년 된 이유. 임예진 남편 최창욱과 결혼 이유?
  
[['desktop.ini', '임예진 이혼_ 남편과 각

...

323
seed : https://www.youtube.com/watch?v=PdT_7ni7scQ&app=desktop

    연애 뭣이 중한디~!
  
[['desktop.ini', '연애 뭣이 중한디_! - Korean (auto-generated).srt']]
/Users/user/Downloads/연애 뭣이 중한디_! - Korean (auto-generated).srt
324
seed : https://www.youtube.com/watch?v=WeCvsk2TrR4&app=desktop

    내 삶의 주인은 나
  
[['desktop.ini', '내 삶의 주인은 나 - Korean (auto-generated).srt']]
/Users/user/Downloads/내 삶의 주인은 나 - Korean (auto-generated).srt
325
seed : https://www.youtube.com/watch?v=d7mkKod8DtQ&app=desktop

    행복하게 산다는 것?
  
[['desktop.ini', '행복하게 산다는 것_ - Korean (auto-generated).srt']]
/Users/user/Downloads/행복하게 산다는 것_ - Korean (auto-generated).srt
326
seed : https://www.youtube.com/watch?v=0cp4dAL5ArE&app=desktop

    마음이 허해요
  
[['desktop.ini', '마음이 허해요 - Korean (auto-generated).srt']]
/Users/user/Downloads/마음이 허해요 - Korean (auto-generated).srt
327
seed : https://www.youtube.com/watch?v=cIAr7tgJ7rs&app=desktop

    50년 삶이 한심해요
  
[['50년 삶이 한심해요 - Korean (auto-generated).srt', 'desktop.ini']]
/Users/us

seed : https://www.youtube.com/watch?v=lW1Z-vvYJBY&app=desktop
366
seed : https://www.youtube.com/watch?v=LWKlVTczFrs&app=desktop
367
seed : https://www.youtube.com/watch?v=T1zXXMOu9Hc&app=desktop

    김미진의 최신 설교 ● 목표를 세우면 행동에 변화를 주어 이룰 수 있겠다 ● 재정강의 2019
  
[['desktop.ini', '김미진의 최신 설교 ● 목표를 세우면 행동에 변화를 주어 이룰 수 있겠다 ● 재정강의 2019 - Korean.srt']]
/Users/user/Downloads/김미진의 최신 설교 ● 목표를 세우면 행동에 변화를 주어 이룰 수 있겠다 ● 재정강의 2019 - Korean.srt
368
seed : https://www.youtube.com/watch?v=_gv1INPnS4U&app=desktop
369
seed : https://www.youtube.com/watch?v=cAJ-htKv744&app=desktop

    김미진 간사 재정강의 2019 ● 절망 속에도 희망은 있다 ● 자기계발
  
[['desktop.ini', '김미진 간사 재정강의 2019 ● 절망 속에도 희망은 있다 ● 자기계발 - Korean.srt']]
/Users/user/Downloads/김미진 간사 재정강의 2019 ● 절망 속에도 희망은 있다 ● 자기계발 - Korean.srt
370
seed : https://www.youtube.com/watch?v=ffCRJDXI7uw&app=desktop
371
seed : https://www.youtube.com/watch?v=m1-PURYR41o&app=desktop

    오늘을 바꿔서 운명을 바꾸자
  
[['desktop.ini', '오늘을 바꿔서 운명을 바꾸자 - Korean.srt']]
/Users/user/Downloads/오늘을 바꿔

405
seed : https://www.youtube.com/watch?v=zWFJPOPRyTg&app=desktop

    [법륜스님의 즉문즉설 제 1518회] 40대 실업, 어떻게 살아야 할까요?
  
[['desktop.ini', '[법륜스님의 즉문즉설 제 1518회] 40대 실업, 어떻게 살아야 할까요_ - Korean (auto-generated).srt']]
/Users/user/Downloads/[법륜스님의 즉문즉설 제 1518회] 40대 실업, 어떻게 살아야 할까요_ - Korean (auto-generated).srt
406
seed : https://www.youtube.com/watch?v=nyhOAcEoTrk&app=desktop

    제784회 남편의 폭언하는 습성 때문에 힘들어요
  
[['desktop.ini', '제784회 남편의 폭언하는 습성 때문에 힘들어요 - Korean (auto-generated).srt']]
/Users/user/Downloads/제784회 남편의 폭언하는 습성 때문에 힘들어요 - Korean (auto-generated).srt
407
seed : https://www.youtube.com/watch?v=vK_RLnrVYUg&app=desktop

    제620회 남편이 새벽에 귀가 합니다. 어떻게 기도를 해야할까요?
  
[['desktop.ini', '제620회 남편이 새벽에 귀가 합니다. 어떻게 기도를 해야할까요_ - Korean (auto-generated).srt']]
/Users/user/Downloads/제620회 남편이 새벽에 귀가 합니다. 어떻게 기도를 해야할까요_ - Korean (auto-generated).srt
408
seed : https://www.youtube.com/watch?v=JCndjrM0iAY&app=desktop

    제530회 남편의 사업 실패, 경매 그리고 이혼 후 아이들을 남편에게 보내고 나니 괴롭습니다
  
[['desktop.ini', '제53

444
seed : https://www.youtube.com/watch?v=iJQoTL4_BNQ&app=desktop

    전원주택 절대 사거나 짓지마라? 그럼 어떻게? 전원생활 3년차의 이유있는 권유
  
[['desktop.ini', '전원주택 절대 사거나 짓지마라_ 그럼 어떻게_ 전원생활 3년차의 이유있는 권유 - Korean (auto-generated).srt']]
/Users/user/Downloads/전원주택 절대 사거나 짓지마라_ 그럼 어떻게_ 전원생활 3년차의 이유있는 권유 - Korean (auto-generated).srt
445
seed : https://www.youtube.com/watch?v=lnQMkFT1PNo&app=desktop

    비오는 날 10분이면 충분한 ‘방충망 청소비법’
  
[['desktop.ini', '비오는 날 10분이면 충분한 ‘방충망 청소비법’ - Korean (auto-generated).srt']]
/Users/user/Downloads/비오는 날 10분이면 충분한 ‘방충망 청소비법’ - Korean (auto-generated).srt
446
seed : https://www.youtube.com/watch?v=PWM062MEIME&app=desktop

    현관에 '이런 물건' 두면 재물운이 막힌다!
  
[['desktop.ini', "현관에 '이런 물건' 두면 재물운이 막힌다! - Korean (auto-generated).srt"]]
/Users/user/Downloads/현관에 '이런 물건' 두면 재물운이 막힌다! - Korean (auto-generated).srt
447
seed : https://www.youtube.com/watch?v=T5KuI_qUCm8&app=desktop
448
seed : https://www.youtube.com/watch?v=YXIf81rPQE4&app=desktop
449
seed : https://www.youtube.com/watch?v=D

477
seed : https://www.youtube.com/watch?v=lB9jEW6EGns&app=desktop

    진짜 신기한 고무줄 연결 마술  해법 마술배우기
  
[['desktop.ini', '진짜 신기한 고무줄 연결 마술 해법 마술배우기 - Korean (auto-generated).srt']]
/Users/user/Downloads/진짜 신기한 고무줄 연결 마술 해법 마술배우기 - Korean (auto-generated).srt
478
seed : https://www.youtube.com/watch?v=Ld94d_Imopw&app=desktop

    [프로의 한수] 약 빨고 두나...미친척 차로 졸을 치겠습니다. (장기)
  
[['desktop.ini', '[프로의 한수] 약 빨고 두나...미친척 차로 졸을 치겠습니다. (장기) - Korean (auto-generated).srt']]
/Users/user/Downloads/[프로의 한수] 약 빨고 두나...미친척 차로 졸을 치겠습니다. (장기) - Korean (auto-generated).srt
479
seed : https://www.youtube.com/watch?v=0Eoya9hQLuM&app=desktop

    [레전드 명경기] 프로의 패기..깔끔하게 차를 주고 갑시다?? (장기)
  
[['desktop.ini', '[레전드 명경기] 프로의 패기..깔끔하게 차를 주고 갑시다__ (장기) - Korean (auto-generated).srt']]
/Users/user/Downloads/[레전드 명경기] 프로의 패기..깔끔하게 차를 주고 갑시다__ (장기) - Korean (auto-generated).srt
480
seed : https://www.youtube.com/watch?v=gfZhjNoDAdg&app=desktop

    [프로의 한수] 위기를 기회로...회심의 일격??? 꼼수 같은디..(장기)
  
[['desktop.ini', '[프로의 한수] 

In [17]:
youtube(n = 10, seed = "https://www.youtube.com/watch?v=MHd93E94kQc&app=desktop") # seed : 쌍튜브 

0
seed : https://www.youtube.com/watch?v=MHd93E94kQc&app=desktop
1
seed : https://www.youtube.com/watch?v=59lib73KEiE&app=desktop

    쌍튜브] 쌍베의 특수능력 '조륜안' - 카트라이더
  
[['desktop.ini', "쌍튜브] 쌍베의 특수능력 '조륜안' - 카트라이더 - Korean (auto-generated).srt"]]
/Users/user/Downloads/쌍튜브] 쌍베의 특수능력 '조륜안' - 카트라이더 - Korean (auto-generated).srt
2
seed : https://www.youtube.com/watch?v=Nz5UUWA1pMg&app=desktop

    쌍튜브] 블리자드가 쌍베에게 독을 탓다.. - 카트라이더
  
[['desktop.ini', '쌍튜브] 블리자드가 쌍베에게 독을 탓다.. - 카트라이더 - Korean (auto-generated).srt']]
/Users/user/Downloads/쌍튜브] 블리자드가 쌍베에게 독을 탓다.. - 카트라이더 - Korean (auto-generated).srt
3
seed : https://www.youtube.com/watch?v=uz3hYOKwoSo&app=desktop
4
seed : https://www.youtube.com/watch?v=N8PAmFrfSuo&app=desktop

    할로윈 인 더 와나나(견자희,형독)
  
[['desktop.ini', '할로윈 인 더 와나나(견자희,형독) - Korean (auto-generated).srt']]
/Users/user/Downloads/할로윈 인 더 와나나(견자희,형독) - Korean (auto-generated).srt
5
seed : https://www.youtube.com/watch?v=EcyanqiEMUc&app=desktop

    탬탬버린님 인성 폭로(with 김진우님)
  
[['des

In [18]:
con.close()